## Procesamiento
1. En el dataset de Aeropuertos (A), se debe generar una columna llamada 'elevation_name' que contenga datos cualitativos indicando la elevación de cada aeropuerto. Esta columna se completará con las palabras: "bajo", "medio" o "alto", según la elevación de cada aeropuerto. Para definir estos grupos, se utilizarán los siguientes valores:
######
1. bajo: aeropuertos con elevación menor o igual a 131 ft.
2. medio: aeropuertos con elevación mayor que 131 ft y menor o igual a 903 ft.
3. altos: aeropuertos con elevación mayor a 903 ft.

Además, se realizará otra modificación en este dataset, consistente en agregar una columna llamada 'prov_name', donde se incluirá el nombre de la provincia correspondiente a cada aeropuerto. Esta información se obtendrá consultando los nombres de las ciudades en el dataset (E).


In [1]:
import csv
import pathlib
import dataset_module

In [7]:
# Database design
elevation_ft_field = 6

read_airports = pathlib.Path('../datasets/original_datasets/ar-airports.csv')
write_airports = pathlib.Path('../datasets/custom_datasets/ar-airports.csv')

with(read_airports.open(mode='r', newline='', encoding='utf-8') as read_airports,
     write_airports.open(mode='w', newline='', encoding='utf-8') as write_airports):
    original_airports = csv.reader(read_airports)
    custom_airports = csv.writer(write_airports)
    #Create the header of the file, including the "elevation_name" and "prov_name" fields
    header = next(original_airports)
    header.append("elevation_name")
    header.append("prov_name")
    #Write header in custom dataset
    custom_airports.writerow(header)
    #Append elevation_name and prov_name for each airport
    for line in original_airports:
        #Include elevation to the list: elevation_ft = empty then elevation_name = empty
        line.append(dataset_module.include_elevation(line))
        #Include province to the list
        line.append(dataset_module.include_prov(line))
        #Write airport in custom dataset
        custom_airports.writerow(line)

2. En el dataset de Conectividad (B), se realizará un reemplazo en las celdas que contengan el carácter '-' con la palabra 'NO'.

Para el mismo dataset se debe generar una nueva columna denominada 'posee_conectividad', la misma puede tomar uno de dos valores posibles: SÍ o NO. 

El valor será NO si todos los campos ADSL, CABLEMODEM, DIALUP, FIBRAOPTICA, SATELITAL, WIRELESS, TELEFONIAFIJA, 3G y 4G poseen el valor --. Caso contrario el valor será SI

In [8]:
read_dataset = pathlib.Path('../datasets/original_datasets/Conectividad_Internet.csv')
write_dataset = pathlib.Path('../datasets/custom_datasets/Conectividad_Internet.csv')

with(read_dataset.open(mode='r', encoding='utf-8') as read_file,
     write_dataset.open(mode='w', encoding='utf-8', newline='') as write_file):
    
    writer = csv.writer(write_file, delimiter=',')
    writer.writerow(['Provincia', 'Partido', 'Localidad', 'Poblacion', 'ADSL', 'CABLEMODEM', 'DIALUP', 'FIBRAOPTICA', 'SATELITAL', 'WIRELESS', 
                     'TELEFONIAFIJA', '3G', '4G', 'link', 'Latitud', 'Longitud', 'posee_conectividad'])

    reader = csv.DictReader(read_file, delimiter=',')
    
    lines = []
    for line in reader:
        line = dataset_module.has_connectivity(line)
        line = dataset_module.hyphen_replace(line)
        lines.append(line)

    for line in lines:
        writer.writerow(line.values())

3. En el dataset de Lagos (C), se creará una nueva columna llamada 'Sup Tamaño' que contendrá datos cualitativos indicando el tamaño de cada lago en función de su superficie en kilómetros cuadrados (km²). Esta columna se completará con las palabras: "chico", "medio" o "grande", según los siguientes criterios:
#####
1. Lagos con una superficie menor o igual a 17 km² serán clasificados como "chico".
2. Lagos con una superficie mayor que 17 km² y menor o igual a 59 km² serán clasificados como "medio".
3. Lagos con una superficie mayor a 59 km² serán clasificados como "grande".

Además de transformar el campo de coordenadas actual, que sigue el formato estándar de grados, minutos y segundos (GMS), por ejemplo, 42°9'3"S 71°38'59"O, en dos campos separados para la latitud y longitud, añade dos nuevos campos para representar la latitud y longitud en formato de grados decimales (GD). Asegúrate de proporcionar la conversión correcta de GMS a GD para ambos campos. En el ejemplo mencionado el valor resultante debería ser -42.150833 para la latitud y -71.649722 para la longitud.


In [3]:
# INDICE DATA LAGOS
NOMBRE = 0
UBICACION = 1
SUPERFICIE = 2
PROFUNDUDAD_MAXIMA = 3
PROFUNDIDAD_MEDIA = 4
CORDENADAS = 5

read_dataset = pathlib.Path('../datasets/original_datasets/lagos_arg.csv')
write_dataset = pathlib.Path('../datasets/custom_datasets/lagos_arg.csv')

with(read_dataset.open(mode='r', encoding='utf-8') as read_file, 
        write_dataset.open(mode='w', newline="", encoding='utf-8') as write_file):

    reader = csv.reader(read_file)
    writer = csv.writer(write_file)
    # Me guardo la primera fila
    header = next(reader)
    # Elimino columna "Coordenadas"
    header.pop()
    # Le agrego al header las columna "Latitud" - "Longitud" - "Sup tamaño"
    # uso .extend() para poder agregar todo en una linea
    header.extend(["Latitud", "Longitud", "Sup tamaño"])
    writer.writerow(header)

    # Recorro fila por fila la data, y modifico lo necesario
    for line in reader:
        # Cordenadas (GMS a GD)
        # Separo en latitud y longitud
        lat_long = line[CORDENADAS].split(" ")
        lat_GD = dataset_module.convert_GMS_coordinate(lat_long[0])
        long_GD = dataset_module.convert_GMS_coordinate(lat_long[1])
        # Elimino la columna de datos de las coordenadas en GMS
        line.pop()
        # Agrego las columnas latitud y longitud
        line.extend([lat_GD, long_GD])

        # Dependiendo de su superficie agrego columna con tamaño
        # "Chico" para sup<=17,"medio" 17<sup<=57 y "grande" >57
        line.append('chico' if int(line[SUPERFICIE]) <= 17 else 'medio' if int(line[SUPERFICIE]) <= 59 else 'grande')
        writer.writerow(line)

4. Para el dataset de Población del censo 2022 (D) reemplazar los valores "///" y "-" por cero en los campos que corresponda.

Además agregar un nuevo campo que tenga el porcentaje de población en situación de calle. Tener en cuenta el total general (NO tener en cuenta los totales por sexo registrado al nacer).

In [2]:
#Indice data censo
total_poblacion = 1
poblacion_en_situacion_de_calle = 4

read_dataset = pathlib.Path('../datasets/original_datasets/c2022_tp_c_resumen_adaptado.csv')
write_dataset = pathlib.Path('../datasets/custom_datasets/c2022_tp_c_resumen_adaptado.csv')

with(read_dataset.open(mode='r',newline="", encoding='utf-8') as read_file,
     write_dataset.open(mode='w',newline="", encoding='utf-8') as write_file):
    
    reader = csv.reader(read_file)
    writer = csv.writer(write_file)
    #Creo el header y agrego la columna 'Porcentaje de población en situación de calle'
    header= next(reader)
    header.append('Porcentaje de población en situación de calle')
    writer.writerow(header)

    for line in reader:
        #reemplazo en la linea '///' o '-' por 0
        lines = dataset_module.replace_values(line)
        #obtengo el porcentaje de poblacion en situacion de calle y lo agrego a la lista        
        percentage= dataset_module.calculate_percentage(lines[total_poblacion],lines[poblacion_en_situacion_de_calle])
        lines.append(percentage)
        writer.writerow(lines)